In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress

In [2]:
calib_df = pd.read_csv("calib.csv")
curr = calib_df["Calib"].to_numpy()
field = calib_df["Field"].to_numpy()
field = field/1000

fig, ax = plt.subplots(figsize=(12,9))
ax.plot(curr, field, "bo-")
ax.set_xlabel("Coil Current (A)", fontsize=15)
ax.set_ylabel("Magnetic Field (kG)", fontsize=15)
ax.grid()
ax.set_title("Calibration : magnetic field vs coil current", fontsize=18)
plt.savefig("calib.jpeg", bbox_inches="tight", dpi=130)
plt.close()

In [3]:
n05a, n205ma, p1a, p481ma = pd.read_csv("n0.5a.csv"), pd.read_csv("n2.05ma.csv"), pd.read_csv("p1a.csv"), pd.read_csv("p4.81ma.csv")
dfs = [n05a, p1a, n205ma, p481ma]
fnames = ["n05a", "p1a", "n205ma", "p481ma" ]
titles = [r"$0.5\,A$", r"$1.0\,A$", r"$2.05\,mA$", r"$4.81\,mA$"]
titles2 = ["n-type Ge", "p-type Ge", "n-type Ge", "p-type Ge"]
slopes= []
i = 0
for df in dfs:
    if (df.columns[0] == "probecurrent"):
        probecurrent = df["probecurrent"].to_numpy()
        hallvoltage = df["hallvoltage"].to_numpy()
        fig, ax = plt.subplots(figsize=(12,9))
        ax.plot(probecurrent, hallvoltage, "bo-")
        ax.set_xlabel("Probe Current (mA)", fontsize=15)
        ax.set_ylabel("Hall Voltage (mV)", fontsize=15)
        ax.set_title(f"Hall voltage vs Probe current ({titles2[i]}, Coil current = {titles[i]})", fontsize=18)
        ax.grid()
        plt.savefig(fnames[i] + ".jpeg", bbox_inches="tight", dpi=120)
        plt.close()
        slopes.append(linregress(probecurrent, hallvoltage).slope)
        i = i + 1
    else:
        field = df["Field"].to_numpy()
        hallvoltage = df["hallvoltage"].to_numpy()
        fig, ax = plt.subplots(figsize=(12,9))
        ax.plot(field, hallvoltage, "bo-")
        ax.set_xlabel("Magnetic field (kG)", fontsize=15)
        ax.set_ylabel("Hall Voltage (mV)", fontsize=15)
        ax.set_title(f"Hall voltage vs Magnetic field ({titles2[i]}, Probe current = {titles[i]})", fontsize=18)
        ax.grid()
        plt.savefig(fnames[i] + ".jpeg", bbox_inches="tight", dpi=120)
        plt.close()
        slopes.append(linregress(field, hallvoltage).slope)
        i = i + 1

In [4]:
hcoeff = []
z = 5e-2
rho = [10, 5]
H = [848.5, 1776]
curr = [205, 481]
e = 1.602e-19
i = 0
for i in range(0,2):
    hcoeff.append(slopes[i] * z/H[i])
for i in range(0, 2):
    hcoeff.append(slopes[i+2] * z/curr[i])

for i in range(0,4):
    if(i<=1):
        print(f"Hall coeff is {hcoeff[i]} cm^3 C^-1")
    else:
        print(f"Hall coeff is {hcoeff[i]} cm^3 C^-1")
nhall = (hcoeff[0] + hcoeff[2])/2
phall = (hcoeff[1] + hcoeff[3])/2
print(f"Avg n type hall coeff = {nhall} cm^3 C^-1")
print(f"Avg p type hall coeff = {phall} cm^3 C^-1")
print(f"n mobility = {nhall * 1/rho[0]} cm^2 V^-1 s^-1")
print(f"p mobility = {phall * 1/rho[1]} cm^2 V^-1 s^-1")
print(f"n carrier concentration = {1/(nhall * e)} cm^-3")
print(f"p carrier concentration = {1/(phall * e)} cm^-3")

Hall coeff is 0.001003684000148349 cm^3 C^-1
Hall coeff is 0.0002284844941094941 cm^3 C^-1
Hall coeff is 0.0036878131971385337 cm^3 C^-1
Hall coeff is 0.0017484154468927055 cm^3 C^-1
Avg n type hall coeff = 0.002345748598643441 cm^3 C^-1
Avg p type hall coeff = 0.0009884499705010998 cm^3 C^-1
n mobility = 0.00023457485986434412 cm^2 V^-1 s^-1
p mobility = 0.00019768999410021997 cm^2 V^-1 s^-1
n carrier concentration = 2.6610683075940466e+21 cm^-3
p carrier concentration = 6.315137275251973e+21 cm^-3
